---
title: "Data Collection"
format:
    html: 
        code-fold: false
---

## Overview

The goal of this section is to collect and consolidate data to build a comprehensive dataset that captures factors influencing negative high school outcomes in New York City. This includes data on high school offerings, quality metrics, and socioeconomic characteristics.


### Goals
* Compile relevant data from multiple sources to understand key drivers behind poor high school outcomes.
* Create a unified dataset that combines academic quality indicators, attendance statistics, and socioeconomic factors for further analysis.

### Motivation
High school outcomes, such as chronic absenteeism, dropping out, and low college persistence rates, can significantly impact students futures. By identifying the underlying factors driving these outcomes, this analysis seeks to provide actionable insights that could inform policies or interventions aimed at improving educational equity and student success in New York City high schools.

### Objectives
* Aggregate and map data from various sources, including high school quality reports and Census data, into one unified dataset.
* Ensure compatibility between datasets by creating shared columns through location metrics, for example ZIP code.
* Lay the groundwork for further analysis, enabling the exploration of correlations between school quality, socioeconomic factors, and student outcomes.

## Methods

### American Community Survey API
In the following code, I first used the requests library to retrieve the Census data from the American Community Survey. The params include get and for, which specify the columns of interest and to pull data at the zip code level, respectively. Chat GPT was used to brainstorm potentional relevant columns to pull from the 10,000+ columns that would satisfy the project objectives [@gpt4o_code_gen]. I stored this in a pandas dataframe and renamed the columns to ensure readability. Finally, I exported this dataframe to a csv file to prepare for data cleaning.

In [2]:
import requests
import pandas as pd


# base url + specifiy params
url = "https://api.census.gov/data/2022/acs/acs5/profile"
params = {
    "get": "DP02_0060PE,DP02_0068PE,DP02_0114PE,DP02_0072PE,DP02_0094PE,DP02_0154PE,DP03_0062E,DP03_0074PE",
    "for": "zip code tabulation area:*",
    "key": "1c6835368d6cc1f7472ed2e8a39e07ee7e9d1cd6"
}

response = requests.get(url, params=params)

# check the response
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])

    df.rename(columns={
    "DP02_0060PE": "Percent No High School (25+)",
    "DP02_0068PE": "Percent Bachelor's Degree or Higher (25+)",
    "DP02_0114PE": "Percent Language Other Than English at Home",
    "DP02_0072PE": "Percent Population with Disabilities",
    "DP02_0094PE": "Percent Foreign-Born Population",
    "DP02_0154PE": "Percent Households with Broadband Internet",
    "DP03_0062E": "Median Household Income",
    "DP03_0074PE": "Percent Households on SNAP/Food Stamps"
    }, inplace=True)

    print(df)
else:
    print(f"Error: {response.status_code} - {response.text}")

      Percent No High School (25+) Percent Bachelor's Degree or Higher (25+)  \
0                             None                                      None   
1                             None                                      None   
2                             None                                      None   
3                             None                                      None   
4                             None                                      None   
...                            ...                                       ...   
33769                          0.0                                      64.0   
33770                          3.0                                      17.0   
33771                          0.9                                       9.2   
33772                          0.0                                       0.0   
33773                          1.7                                      18.0   

      Percent Language Other Than Engli

Export to csv

In [3]:
df.to_csv('../../data/raw-data/ACS_data.csv')

### NYC Open Data API
This data on graduation rate outcomes is taken from NYC Open Data. This file contains the dropout rates of interest. This code is adapted from the API documentation [@noauthor_2005-2019_nodate]. The library Socrata from sodapy is used to retrieve the data. The package returns the JSON data to a Python list of dictionaries for easy conversion to a pandas dataframe.

In [4]:
from sodapy import Socrata

# unauthenticated client works with public data sets

client = Socrata("data.cityofnewyork.us", None)
results = client.get("mjm3-8dw8", limit=321002)

# convert to pandas df
results_df = pd.DataFrame.from_records(results)

Export to csv

In [32]:
results_df.to_csv('../../data/raw-data/dropout_data.csv')

### Other Datasets

To gather data on the quality of each high school in New York City, including some of my targets of interest, such as chronic absenteeism and college persistence, I downloaded an xlsx dataset from NYC InfoHub.

[Download the 2022-23 High School Quality Report]('../../data/raw-data/202223-hs-sqr-results.xlsx')

Additionally, to map the quality data to Census data, I used an NYC High School Directory file available on the InfoHub site. This file provided ZIP codes, which enabled me to join the quality data with Census data.

[Download the mapping file]('../../data/mapping/2021_DOE_High_School_Directory.csv')


## Takeaways

### Summary

This section focuses on collecting and integrating data to build a comprehensive dataset that explores the factors influencing negative high school outcomes in New York City. Using data from NYC InfoHub and Census mapping, I was able to create a unified and actionable dataset that will provides insights for improving educational equity and student success. This foundational work enables deeper analysis to inform strategies and interventions for better high school outcomes.

### Challenges
One major challenge I faced was finding data that could be aggregated into my dataset. The NYC high school quality data is organized by school name and DBN, a unique New York identifier that combines the district, borough, and NYC DOE school number. However, many other datasets, such as nationwide ones that I was interested in, did not include these identifiers, as they are specific to New York. As a result, I had to spend considerable time finding data with compatible aggregations, which somewhat limited my options. Eventually, I was able to find a mapping of DBN to ZIP code, which allowed me to incorporate census data into my school data, adding a few socioeconomic factors.

For future work, I would be interested in scaling this project to include all of New York State or even nationwide data. Currently, my analysis is limited to New York City high schools, resulting in a dataset of only about 500 rows, which is relatively small. Expanding to a larger dataset with more schools would likely enable the creation of more robust datasets and models.

### Conclusion and Future Steps

By aggregating data from three different datasets, a comprehensive dataset is created to evaluate various features that may impact our target variables. This analysis helps us better understand how high school quality and socioeconomic factors influence high school outcomes in New York City. Next steps involve cleaning and exploring this data in order to prepare it for the modeling phase.